## 🌀 Quick Start Guide: It's all starting to unravel!

First run `pip install unravelsports` if you haven't already!


-----


In [ ]:
%pip install unravelsports --quiet

### 1. Processing Data

1. Load [Kloppy](https://github.com/PySport/kloppy) dataset. See [in-depth Tutorial](1_kloppy_gnn_train.ipynb) on how do processes multiple match files.
2. Convert to Graph format using `GraphConverter`
3. Create dataset for easy processing with [Spektral](https://graphneural.network/) using `CustomSpektralDataset`

In [1]:
from unravel.soccer import GraphConverter
from unravel.utils import CustomSpektralDataset

from kloppy import skillcorner

from unravel.utils import dummy_labels

# Load Kloppy dataset
kloppy_dataset = skillcorner.load_open_data(
    match_id=4039,
    include_empty_frames=False,
    limit=500,  # limit to 500 frames in this example
)

# Initialize the Graph Converter, with dataset, labels and settings
converter = GraphConverter(dataset=kloppy_dataset, labels=dummy_labels(kloppy_dataset))

# Compute the graphs and add them to the CustomSpektralDataset
dataset = CustomSpektralDataset(graph=converter.to_spektral_graphs())

Processing frames: 100%|██████████| 500/500 [00:00<00:00, 561.72it/s]

Loading 477 graphs into CustomSpektralDataset...
Loading 477 graphs into CustomSpektralDataset...


### 2. Split Data

Split the dataset with the built in `split_test_train_validation` method.

In [2]:
from spektral.data import DisjointLoader

train, test, val = dataset.split_test_train_validation(
    split_train=4, split_test=1, split_validation=1, random_seed=42
)

#### 2. Compile Model

1. Initialize the `CrystalGraphClassifier`.
2. Compile the model with a loss function, optimizer and your preferred metrics.

In [3]:
from unravel.classifiers import CrystalGraphClassifier

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, BinaryAccuracy

model = CrystalGraphClassifier()

model.compile(
    loss=BinaryCrossentropy(), optimizer=Adam(), metrics=[AUC(), BinaryAccuracy()]
)

### 3. Fit Model

1. Create a [`DisjointLoader`](https://graphneural.network/loaders/#disjointloader) for training and validation sets.
2. Fit the model. Note: set `use_multiprocessing=True` to speed up training significantly. 

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
epochs = 10

loader_tr = DisjointLoader(train, batch_size=batch_size)
loader_va = DisjointLoader(val, epochs=1, shuffle=False, batch_size=batch_size)

model.fit(
    loader_tr.load(),
    epochs=epochs,
    steps_per_epoch=loader_tr.steps_per_epoch,
    use_multiprocessing=True,
    validation_data=loader_va.load(),
    callbacks=[EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)],
)

Epoch 1/10


/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


10/10 [==============================] - 2s 26ms/step - loss: 16.1988 - auc: 0.4597 - binary_accuracy: 0.4560 - val_loss: 7.9675 - val_auc: 0.5101 - val_binary_accuracy: 0.4625
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 8.1257 - auc: 0.5055 - binary_accuracy: 0.5283 WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 3 batches). You may need to use the repeat() function when building your dataset.


10/10 [==============================] - 0s 9ms/step - loss: 8.1257 - auc: 0.5055 - binary_accuracy: 0.5283
Epoch 3/10
10/10 [==============================] - 0s 8ms/step - loss: 3.8924 - auc: 0.5408 - binary_accuracy: 0.5535
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 3.6436 - auc: 0.4814 - binary_accuracy: 0.5000
Epoch 5/10
10/10 [==============================] - 0s 11ms/step - loss: 2.2543 - auc: 0.4966 - binary_accuracy: 0.4969
Epoch 6/10
10/10 [==============================] - 0s 6ms/step - loss: 1.7266 - auc: 0.4905 - binary_accuracy: 0.4780
Epoch 7/10
10/10 [==============================] - 0s 6ms/step - loss: 1.4315 - auc: 0.5282 - binary_accuracy: 0.5126
Epoch 8/10
10/10 [==============================] - 0s 9ms/step - loss: 1.1989 - auc: 0.5094 - binary_accuracy: 0.4748
Epoch 9/10
10/10 [==============================] - 0s 8ms/step - loss: 0.9823 - auc: 0.5714 - binary_accuracy: 0.5440
Epoch 10/10
10/10 [==============================] - 0s 6

### 4. Evaluate Model Performance

1. Create another `DisjointLoader`, this time for the test set.
2. Evaluate model performance on the test set. This evaluation function uses the `metrics` passed to `model.compile`

Note: Our performance is really bad because we're using random labels, very few epochs and a small dataset.

In [6]:
loader_te = DisjointLoader(test, epochs=1, shuffle=False, batch_size=batch_size)
results = model.evaluate(loader_te.load())

3/3 [==============================] - 0s 22ms/step - loss: 0.7569 - auc: 0.4881 - binary_accuracy: 0.4684


### 5. Predict

1. Use unseen data to predict on. In this example we're using the test dataset.
2. We have to re-create `loader_te` because `DisjointLoader` is a generator.
3. Setting `batch_size` and `use_multiprocessing=True` on prediction helps speed up the prediction

In [7]:
loader_te = DisjointLoader(test, batch_size=batch_size, epochs=1, shuffle=False)
loaded_pred = model.predict(loader_te.load(), use_multiprocessing=True)

loaded_pred

4/4 [==============================] - 0s 11ms/step


array([[0.32064953],
       [0.5187207 ],
       [0.36767447],
       [0.58158153],
       [0.648303  ],
       [0.7250808 ],
       [0.45932224],
       [0.17837673],
       [0.40539873],
       [0.4424542 ],
       [0.30549103],
       [0.4686426 ],
       [0.45274827],
       [0.48409605],
       [0.52468264],
       [0.48941866],
       [0.45071444],
       [0.40024903],
       [0.43896046],
       [0.4752483 ],
       [0.6113058 ],
       [0.51407444],
       [0.63001126],
       [0.4740725 ],
       [0.6078143 ],
       [0.42406863],
       [0.42664722],
       [0.5456678 ],
       [0.6497228 ],
       [0.10468224],
       [0.28870958],
       [0.35573283],
       [0.50675493],
       [0.26084217],
       [0.5045665 ],
       [0.4551955 ],
       [0.2941906 ],
       [0.17870983],
       [0.645881  ],
       [0.44119328],
       [0.5141012 ],
       [0.4944313 ],
       [0.5526908 ],
       [0.3726117 ],
       [0.01953349],
       [0.43424365],
       [0.6119593 ],
       [0.173